#### Read Historical Data Base - read_historical_data_base

In [ ]:
_ref = input_refresh_node   

_file_name = file_name.lower().replace(' ', '_') + '.pkl'
_path_file = path_historical_data_base_pkl
_path = os.path.join(_path_file, _file_name) 
_hist_base = pd.read_pickle( path = _path , compression='infer' ) 
 
result = _hist_base

#### Read Historical Data - read_historical_data_td

In [ ]:
_ref = input_refresh_node   

_conf = historical_data_conf
_versions = historical_data_versions
_sel_version = historical_data_sel_version

result = read_excel_template(_conf, _versions, _sel_version)

#### Base Historical Data Concat Task - base_historical_data_concat_task

In [ ]:
import os

try: 

    # Historical Base
    _hist_base = read_historical_data_base.copy()
    
    # #New base
    _new_base= read_historical_data_td.copy()
    _time_add = _new_base["Ano-Mês"].unique()
    
    # Drop time that are in both bases, keep always the new one
    _hist_base = _hist_base[~_hist_base["Ano-Mês"].isin(_time_add)]
    _time_col = [c for c in _hist_base["Ano-Mês"].unique() if c not in _new_base["Ano-Mês"].unique() ]
    _hist_base = _hist_base[_hist_base["Ano-Mês"].isin(_time_col)]
    
    #Missing Columns in the new database
    _missing_columns = [c for c in _hist_base if c not in _new_base]
    
    #Fill the old database with np.nan, then when the task have already runned, add the missing columns with the hierarchy.
    if len(_missing_columns)>0:
        for i in _missing_columns:
            _new_base[i] = np.nan
    
    #Order with the same columns
    _order_cols = [c for c in _hist_base]
    _new_base = _new_base[_order_cols]
    
    #Concat
    _df = pd.concat([_hist_base,_new_base], axis=0)
    _df = _df[ (_df['Ano-Mês'] != '-') | (_df['Fábrica'] != 0 ) ]
    _df = _df.sort_values(by=["Ano-Mês"], ascending=True).reset_index(drop=True)
    
    result = _df

except:
    # Historical Base
    _hist_base = read_historical_data_base.copy()
    _df = _hist_base
    result = _df

#### Inputs Root Path - input_root_path

In [ ]:
result = '/data/models/intercement/Public/IC - Sales and Operations Planning Tool/Inputs'

#### Path Historical Data Base pkl - path_historical_data_base_pkl

In [1]:
result = os.path.join(input_root_path)

NameError: name 'input_root_path' is not defined

File Name Base Historical - file_name

In [ ]:
result = "Historical Data Base"

#### Save Read Historical Data Actual - save_read_historical_data_actual

In [ ]:
import datetime as dt
from datetime import date
   
# _user = pyplan_user['userName']
# _date = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# +'/'+file_name+'__'+_user+'__'+_date + 
_file_name = file_name.lower().replace(' ', '_')

if escolher_node_para_salvar.value == read_historical_data_td.node.title :
    _df = base_historical_data_concat_task.copy()
    result = _df.to_pickle(os.path.join(path_historical_data_base_pkl, _file_name) + '.pkl')

else: 
    _df = read_historical_data_td.copy()
    result = _df.to_pickle(os.path.join(path_historical_data_base_pkl, _file_name) + '.pkl')

#### Tipo de Salvamento - tipo_de_salvamento

In [ ]:
result = pd.Index([read_historical_data_td.node.title, base_historical_data_concat_task.node.title])

#### Escolher Node Para Salvar - escolher_node_para_salvar

In [ ]:
result = pp.selector(tipo_de_salvamento,1,False)

#### Atualizar dados Históricos - atualizar_dados_historicos

In [ ]:
_msg_progressbar = 'Atualizando Dados Históricos'
_msg_text_send_msg = 'Dados Históricos'
_msg_title_send_msg = 'Nova base gerada'
_msg_bgd = 'info'

try: 

    pp.progressbar( 25, message_text=_msg_progressbar, not_level_reverse=_msg_bgd )
    
    pp.progressbar( 50, message_text=_msg_progressbar, not_level_reverse=_msg_bgd )
    
    pp.progressbar( 75, message_text=_msg_progressbar, not_level_reverse=_msg_bgd )
    
    save_read_historical_data_actual
    
    pp.progressbar( 100, message_text=_msg_progressbar, not_level_reverse=_msg_bgd )
    
    pp.send_message( message_text=_msg_text_send_msg, message_title=_msg_title_send_msg, not_level_reverse=_msg_bgd )
    
except:
    _msg_title_send_msg = 'Nova base não gerada'
    _msg_bgd = 'warning'
    pp.send_message( message_text=_msg_text_send_msg, message_title=_msg_title_send_msg, not_level_reverse=_msg_bgd )